In [1]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [5]:
data=pd.read_fwf("C:/Users/cansh/Downloads/dialogs.txt")

In [6]:
print(data.head())

  hi, how are you doing?\ti'm fine. how about yourself? Unnamed: 1 Unnamed: 2  \
0  i'm fine. how about yourself?\ti'm pretty good...           NaN        NaN   
1  i'm pretty good. thanks for asking.\tno proble...           NaN        NaN   
2  no problem. so how have you been?\ti've been g...           NaN        NaN   
3  i've been great. what about you?\ti've been go...           NaN        NaN   
4  i've been good. i'm in school right now.\twhat...           NaN        NaN   

  Unnamed: 3  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  


In [7]:
text=data['hi, how are you doing?\ti\'m fine. how about yourself?'].to_list()

In [8]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(text)
seq = tokenizer.texts_to_sequences(text)
seq[:10]

[[31, 660, 37, 33, 612, 31, 155, 43, 247, 23, 495],
 [31, 155, 43, 247, 23, 495, 28, 169, 20, 37, 13, 2, 99],
 [28, 169, 20, 37, 13, 2, 99, 100, 99, 104, 9, 33, 2],
 [100, 99, 104, 9, 33, 2, 100, 99, 43, 31, 15, 93, 67, 113],
 [100, 99, 43, 31, 15, 93, 67, 113, 9, 93, 8, 2, 36, 4],
 [9, 93, 8, 2, 36, 4, 1, 36, 4, 808],
 [1, 36, 4, 808, 8, 2, 21, 6, 54],
 [8, 2, 21, 6, 54, 18, 86, 18, 5, 48, 149, 1522],
 [18, 86, 18, 5, 48, 149, 1522, 43, 809, 46, 93],
 [43, 809, 46, 93, 141, 2, 135, 85]]

In [9]:
#tokenizer.word_index
X = []
y = []
total_words_dropped = 0

for i in seq:
    if len(i) > 1:
        for index in range(1, len(i)):
            X.append(i[:index])
            y.append(i[index])
    else:
        total_words_dropped += 1

print("Total Single Words Dropped are:", total_words_dropped)

Total Single Words Dropped are: 0


In [10]:
X = tf.keras.preprocessing.sequence.pad_sequences(X)

In [11]:
X.shape

(44317, 28)

In [12]:
y = tf.keras.utils.to_categorical(y)

In [13]:
vocab_size = len(tokenizer.word_index) + 1

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 14),
    tf.keras.layers.LSTM(100, return_sequences=True),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(vocab_size, activation='softmax'),
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 14)          35448     
                                                                 
 lstm (LSTM)                 (None, None, 100)         46000     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 2532)              255732    
                                                                 
Total params: 427,680
Trainable params: 427,680
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.004),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [18]:
model.fit(X, y, epochs=100)

Epoch 1/100
1385/1385 [==============================] - 12s 9ms/step - loss: 1.9933 - accuracy: 0.5007
Epoch 2/100
1385/1385 [==============================] - 12s 9ms/step - loss: 2.0017 - accuracy: 0.4968
Epoch 3/100
1385/1385 [==============================] - 12s 9ms/step - loss: 2.0154 - accuracy: 0.4918
Epoch 4/100
1385/1385 [==============================] - 12s 9ms/step - loss: 1.9968 - accuracy: 0.5009
Epoch 5/100
1385/1385 [==============================] - 12s 9ms/step - loss: 1.9833 - accuracy: 0.5008
Epoch 6/100
1385/1385 [==============================] - 12s 9ms/step - loss: 2.0054 - accuracy: 0.4962
Epoch 7/100
1385/1385 [==============================] - 12s 9ms/step - loss: 2.0211 - accuracy: 0.4978
Epoch 8/100
1385/1385 [==============================] - 12s 9ms/step - loss: 1.9997 - accuracy: 0.4974
Epoch 9/100
1385/1385 [==============================] - 12s 9ms/step - loss: 1.9962 - accuracy: 0.4991
Epoch 10/100
1385/1385 [==============================] - 12s 9m

In [20]:
model.save('asl.h')

In [9]:
vocab_array = np.array(list(tokenizer.word_index.keys()))
vocab_array

array(['i', 'you', 'the', ..., 'planes', 'bless', 'modern'], dtype='<U16')

In [22]:
def make_prediction(text, n_words, top_n=3):
    pred_list=[]
    for i in range(n_words):
        text_tokenize = tokenizer.texts_to_sequences([text])
        text_padded = tf.keras.preprocessing.sequence.pad_sequences(text_tokenize, maxlen=14)

        probabilities = model.predict(text_padded)[0]  # Assuming batch size is 1

        # Get indices of top_n predictions
        top_indices = np.argsort(probabilities)[-top_n:][::-1]

        # Get corresponding words and probabilities
        top_predictions = [(vocab_array[idx - 1], probabilities[idx]) for idx in top_indices]

        print(f"Step {i+1} predictions:")
        for word, probability in top_predictions:
            print(f"Word: {word}, Probability: {probability:.4f}")
            pred_list.append(word)
        # Use the most probable prediction
        prediction = np.argmax(probabilities)
        prediction = str(vocab_array[prediction - 1])

        text += " " + prediction

    return pred_list






In [71]:
make_prediction("",1)

1/1 [==============================] - 0s 22ms/step
Step 1 predictions:
Word: to, Probability: 0.9230
Word: 80, Probability: 0.0303
Word: elected, Probability: 0.0164


['to', '80', 'elected']

In [ ]:
text_list = []
index = 0

while True:
    out = dict[model.output]
    
    if not out:
        break
    
    if index == 0 or text_list[index - 1] != out:  # Check if the output is different from the previous one
        text_list.append(out)
        index += 1
        text_to_predict = ' '.join(text_list)  # Join the text list to form a single string
        make_prediction(text_to_predict, 1)  # Call your make_prediction function
        
    elif model.output == 21:  # Check for specific condition
        text_list.pop()
        index -= 1
        
    elif model.output in [22, 23, 24]:  # Check for specific conditions
        out = return_list[model.output - 22]
        
        if text_list[index - 1] != out:
            text_list.append(out)
            index += 1
            text_to_predict = ' '.join(text_list)
            make_prediction(text_to_predict, 1)

